In [1]:
from IPython.display import Markdown, Latex
from datasets import load_dataset
import numpy as np
import pandas as pd

In [2]:
methods = [
    "danjacobellis/LSDIR_WEBP_q20",
    "danjacobellis/LSDIR_DGML_q6",
    "danjacobellis/LSDIR_RGB_Li_48c_J3_nf8",
    "danjacobellis/LSDIR_stable_diffusion_3_fp16",
    "danjacobellis/LSDIR_RGB_Li_12c_J3_nf8",
]

dimensionality_reduction = [
    1,
    4,
    4,
    12,
    16,
]

names = [
    'WEBP',
    'Cheng2020',
    'WaLLoC',
    'Stable Diffusion 3',
    'WaLLoC',
]

metrics = [
    'PSNR',
    'MSSIM',
    'LPIPS_dB',
    'DISTS_dB',
]

columns = [
    'Name',
    'Dimensionality Reduction',
    'Compression Ratio',
    'Encode',
    'Decode',
] + metrics

In [3]:
data_rows = []
for i_method, method in enumerate(methods):
    ds = load_dataset(method, split='validation')
    row = [names[i_method], dimensionality_reduction[i_method]]
    compression_ratio = 24 / np.mean(ds['bpp'])
    encode_MP_per_second = np.mean(np.array(ds['n_pixels'])/np.array(ds['cpu_encode_time'])/1e6)
    decode_MP_per_second = np.mean(np.array(ds['n_pixels'])/np.array(ds['cpu_decode_time'])/1e6)
    row.append(compression_ratio)
    row.append(encode_MP_per_second)
    row.append(decode_MP_per_second)
    for metric in metrics:
        mean = np.mean(ds[metric])
        row.append(mean)
    data_rows.append(row)

df = pd.DataFrame(data_rows, columns=columns)

df_rounded = df.round(4)
md_table = df_rounded.to_markdown(index=False)
display(Markdown(md_table))

| Name               |   Dimensionality Reduction |   Compression Ratio |   Encode |    Decode |    PSNR |   MSSIM |   LPIPS_dB |   DISTS_dB |
|:-------------------|---------------------------:|--------------------:|---------:|----------:|--------:|--------:|-----------:|-----------:|
| WEBP               |                          1 |             40.6318 |  22.1209 | 2746.8    | 28.2272 |  0.9597 |     5.9426 |    13.1392 |
| Cheng2020          |                          4 |             21.7528 |   0.289  |    0.1395 | 33.7758 |  0.9903 |     8.8248 |    16.9385 |
| WaLLoC             |                          4 |              8.5313 |  13.9753 |    0.47   | 33.5474 |  0.9902 |    11.2465 |    19.2875 |
| Stable Diffusion 3 |                         12 |              6      |   0.1949 |    0.1013 | 20.8806 |  0.8399 |     8.3253 |    13.8466 |
| WaLLoC             |                         16 |             35.1925 |  22.0586 |    0.4657 | 27.4524 |  0.97   |     6.5131 |    13.8514 |

In [4]:
df_rounded = df.round(10)
column_format = 'l' + 'rr' * len(metrics)
latex_table = df_rounded.to_latex(
    index=True,
    float_format="%.3g",
    column_format=column_format,
    multicolumn=True,
    multicolumn_format='c',
    bold_rows=True
)
print(latex_table)

\begin{tabular}{lrrrrrrrr}
\toprule
 & Name & Dimensionality Reduction & Compression Ratio & Encode & Decode & PSNR & MSSIM & LPIPS_dB & DISTS_dB \\
\midrule
\textbf{0} & WEBP & 1 & 40.6 & 22.1 & 2.75e+03 & 28.2 & 0.96 & 5.94 & 13.1 \\
\textbf{1} & Cheng2020 & 4 & 21.8 & 0.289 & 0.139 & 33.8 & 0.99 & 8.82 & 16.9 \\
\textbf{2} & WaLLoC & 4 & 8.53 & 14 & 0.47 & 33.5 & 0.99 & 11.2 & 19.3 \\
\textbf{3} & Stable Diffusion 3 & 12 & 6 & 0.195 & 0.101 & 20.9 & 0.84 & 8.33 & 13.8 \\
\textbf{4} & WaLLoC & 16 & 35.2 & 22.1 & 0.466 & 27.5 & 0.97 & 6.51 & 13.9 \\
\bottomrule
\end{tabular}

